In [63]:
from pyomo.environ import *

m = AbstractModel()
data = DataPortal()

# sets
m.i_op = Set()

# parameters
m.p_capacity = Param(m.i_op)
m.p_demand = Param()
m.p_cost = Param(m.i_op)

# data import
data.load(filename='EBSD_i_op_data.csv', 
          format='set',
          set=m.i_op)

data.load(filename='EBSD_p_demand_data.csv', 
          param=m.p_demand)

data.load(filename='EBSD_param_data.csv', 
          format='table',
          param=(m.p_capacity, m.p_cost),
          index=m.i_op)

# variables
m.v_u = Var(m.i_op, domain=NonNegativeReals)

# objective
def obj_rule(m):
    return summation(m.p_cost, m.v_u)
m.obj = Objective(rule=obj_rule)

# constraints
def c_rule_supplyMeetsDemand(m):
    return sum(m.v_u[op] for op in m.i_op) >= m.p_demand
m.c_supplyMeetsDemand = Constraint(rule=c_rule_supplyMeetsDemand)

def c_rule_uLessThanCap(m, op):
    return m.v_u[op] <= m.p_capacity[op]
m.c_uLessThanCap = Constraint(m.i_op, rule=c_rule_uLessThanCap)


In [64]:
i = m.create_instance(data)
solver = SolverFactory('glpk')
run = solver.solve(i, tee=True)
run.write(num=1)

i.solutions.load_from(results)


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\user1\AppData\Local\Temp\tmp8c6l_154.glpk.raw --wglp C:\Users\user1\AppData\Local\Temp\tmpggrtrsrg.glpk.glp
 --cpxlp C:\Users\user1\AppData\Local\Temp\tmp21gedvcm.pyomo.lp
Reading problem data from 'C:\Users\user1\AppData\Local\Temp\tmp21gedvcm.pyomo.lp'...
5 rows, 4 columns, 7 non-zeros
36 lines were read
Writing problem data to 'C:\Users\user1\AppData\Local\Temp\tmpggrtrsrg.glpk.glp'...
27 lines were written
GLPK Simplex Optimizer, v4.65
5 rows, 4 columns, 7 non-zeros
Preprocessing...
1 row, 3 columns, 3 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   0.000000000e+00 inf =   1.200e+02 (1)
      2: obj =   6.400000000e+03 inf =   0.000e+00 (0)
*     5: obj =   3.600000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0

In [73]:
for v in i.component_data_objects(Var):
  print(str(v), v.value)

v_u[DM1] 0.0
v_u[RES1] 120.0
v_u[GW1] 0.0
